In [1]:
import os
os.chdir("..")
print("Path now: "+os.getcwd())

Path now: c:\Data\DOCS\ME\Berkas S2 ITS\Kuliah\Semester 1\Rekayasa Perangkat Lunak\apps\mlf_classify\modeling


# Import Library


In [5]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import LinearSVC
from sklearn.preprocessing import  LabelEncoder
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
import numpy as np

from src.model import *
from src.preprocessing import *

c:\Data\DOCS\ME\Berkas S2 ITS\Kuliah\Semester 1\Rekayasa Perangkat Lunak\apps\mlf_classify\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
tokenizer = AutoTokenizer.from_pretrained("indobenchmark/indobert-base-p1")
model = AutoModel.from_pretrained("indobenchmark/indobert-base-p1")

In [32]:
# df = pd.read_csv("Dataset/real_training_data.csv", skipinitialspace=True, encoding='unicode_escape')
df = pd.read_csv("Dataset/real_training_data.csv", sep=";", encoding='utf-8')
df['message'] = df['message'].astype(str)
df['message'] = df['message'].str.lower()
X = df["message"].values

labels = df['kategori'].values

# print(labels)

In [37]:
X = batch_clean(X)
labels = batch_clean(labels)
print(X)

['hi widya we believe that the request has been fulfilled hence we would go ahead and resolve this ticket in case you need any further assistance on it please respond to this email and the ticket will be reopened ticket httpshelpdeskteamstreamzcomatickets64956 regards satish sahu teamstreamz support for any support please feel free to write us at supportteamstreamzcom your feedback helps us give you our best! on wed 17 apr at 946 am wecare team wrote dear mba irma kami telah berhasil merubah akun dsr screenshot terlampir mohon di review kembali kami telah berhasil mengupload pjp untuk distributor tri motor terlampir invalid data mohon di review kembali terima kasih salam sukses castrol we care from irma amalia sent wednesday april 17 2024 1006 am to castrolwecarewgroupid ningsih widya cc teamstreamz support subject re pjp trimotor april hi we care please help thank you regards irma amalia csu support pt castrol indonesia perkantoran hijau arkadia tower g lt 3 jl tb simatupang kav88 jak

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.2, random_state=42)

In [34]:
# Create data loaders
train_loader = batch_tokenize_data(tokenizer, X_train.tolist())
val_loader = batch_tokenize_data(tokenizer, X_test.tolist())

In [35]:
encoder = LabelEncoder()
y_train_ = encoder.fit_transform(y_train)
y_test_ = encoder.transform(y_test)
y_train_

array([0, 0, 1, ..., 1, 1, 1], dtype=int64)

In [38]:
hidden_train = get_cls_hidden_state_batches(train_loader, model)
hidden_val = get_cls_hidden_state_batches(val_loader, model)

Processing batches: 100%|██████████| 6/6 [17:09<00:00, 171.60s/it]


In [39]:
cls_train = hidden_train[:,0,:]
cls_val = hidden_val[:,0,:]

In [40]:
models= {
    "rf": RandomForestClassifier(),
    "xgb": XGBClassifier(),
    "svm": LinearSVC()
}

for key, model in models.items():
    model.fit(cls_train,y_train_)

c:\Data\DOCS\ME\Berkas S2 ITS\Kuliah\Semester 1\Rekayasa Perangkat Lunak\apps\mlf_classify\.venv\Lib\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Data\DOCS\ME\Berkas S2 ITS\Kuliah\Semester 1\Rekayasa Perangkat Lunak\apps\mlf_classify\.venv\Lib\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [41]:
preds = {key: model.predict(cls_val) for key,model in models.items()}

In [42]:
models_score = compare_model(preds,y_test_)
models_score

c:\Data\DOCS\ME\Berkas S2 ITS\Kuliah\Semester 1\Rekayasa Perangkat Lunak\apps\mlf_classify\modeling\src\model.py:129: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  return pd.concat([results, pd.DataFrame(data_list)], ignore_index=True)


,Model,Accuracy,Precision,Recall,F1 Score,MCC
0,rf,0.952652,0.952657,0.952652,0.952651,0.905307
1,xgb,0.960227,0.960390,0.960227,0.960222,0.920613
2,svm,0.965909,0.966334,0.965909,0.965899,0.932238


In [43]:
import os
import pickle

for key,model in models.items():
    os.makedirs("./model_library", exist_ok=True)
    pickle.dump(model, open(f'./model_library/{key}.pkl', 'wb'))